<a href="https://colab.research.google.com/github/boxiXia/MECE4606-Food-printing/blob/master/Food_printing_starter_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Food printing starter code

## reference for the syringe tip (metal tip)
![alt text](https://images-na.ssl-images-amazon.com/images/I/6192oSnVBxL._SL1006_.jpg "syringe tip reference")
[reference](https://www.amazon.com/dp/B01I1APLIG/?coliid=I1H323R2GGYV4D&colid=1VE48W9F0U468&psc=0&ref_=lv_ov_lig_dp_it)

Some Markdown text with <span style="color:blue">some *blue* text</span>.

## Gcode reference
https://duet3d.dozuki.com/Wiki/Gcode

## trouble shooting wifi connection
In printer, go to console, type in command:

```
M552 S1
M552
```
[reference](https://duet3d.dozuki.com/Wiki/Gcode#Section_M552_Set_IP_address_enable_disable_network_interface)


## Preparation tips
+ use Coconut oil to lubricate syringe plungers.
+ clean the syringe plunger and lead screws after use
+ avoid putting too much material in the syringe so material doesn’t ooze out from the top. If material oozes out, clean up immediately so food doesn’t get on the extruder lead screw threads. 


In [144]:
import numpy as np
from numpy.linalg import norm

In [145]:
D = 15.8 # [mm] inner diameter of the cylinder (syringe)
d = 1.64 # [mm] inner diameter of the syring tip, guage 14

TOOL = { # ratio between extruders
    0:[1.0,0.0], # left extruder
    1:[0.0,1.0], # right extruder
    2:[0.5,0.5] # two extruder mixing, for example
}



gcode_para = { # G code parameters
    'D':D, # mm, diameter of the cylinder
    'd':d, # mm, diameter of the extruder
    'applied_percentage':1, # A ratio factor that controls the extruding ratio.
    'TOOL':TOOL,
    'feedrate_move':1000,
    'feedrate_quickmove':2000,
    }

In [146]:
gcode_initial = ''.join([
    'G92 Z0;set current z as z=0\n',
    'G1 F{0}; set feedrate\n'.format(gcode_para['feedrate_move']),
    'G91; Set to Relative Positioning \n'])

In [147]:
print(gcode_initial)

G92 Z0;set current z as z=0
G1 F1000; set feedrate
G91; Set to Relative Positioning 



In [148]:
def ExtruderRatio(gcode_para):
    """A function that calculates the extruder ratio. applied_ratio
    = (D/d)^2*applied_percentage
    @boxiXia;@JacobJ77
    """
    D,d,applied_percentage = gcode_para['D'],gcode_para['d'],gcode_para['applied_percentage']    
    ratio = d ** 2 / (D ** 2) # extruder ratio
    #applied_percentage: applied percentage for extruding
    applied_ratio = ratio * applied_percentage
    return applied_ratio

In [149]:
extruder_ratio = ExtruderRatio(gcode_para)

In [150]:
extruder_ratio

0.010773914436788974

In [151]:
path_preparation = np.array([
    [1,1,0],
    [-1,-1,0],
    [1,1,0],
    [-1,-1,0],
    [1,1,0],
    [-1,-1,0],
    [1,1,0],
    [-1,-1,0],
    [1,1,0],
    [-1,-1,0],
    [1,1,0],
    [-1,-1,0],
    [1,1,0],
    [-1,-1,0],
    [1,1,0],
    [-1,-1,0],
    [1,1,0]
])*8

path = np.array([

    [0.5,0.5,0],
    [1,0,0],
    [0,1,0],
    [-1,0,0],
    [0,-1,0],
])*50 # x,y,z relative movement

path = np.concatenate((path_preparation,path),axis=0)

e_path = norm(path,axis=1)*extruder_ratio # extruder path

In [152]:
def PrintPath(path,e_path,gcode_para,tool_id = 0):
    
    mix_ratio = gcode_para['TOOL'][tool_id]
    gcode_list = [None for k in range(len(path))]
    for i in range(len(path)):
        gcode_list[i] = 'G1 X{0:0.5f} Y{1:0.5f} Z{2:0.5f} E{3:0.5f}:{4:.5f} F{5:.5f}\n'.\
                format(path[i,0],
                       path[i,1],
                       path[i,2],
                       e_path[i] * mix_ratio[0],
                       e_path[i] * mix_ratio[1],
                       gcode_para['feedrate_move'])
    gcode_list.insert(0,'T2;set tool\n')
    return ''.join(gcode_list)

In [153]:
g_code_path = PrintPath(path,e_path,gcode_para)

In [154]:
print(g_code_path)

T2;set tool
G1 X8.00000 Y8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X-8.00000 Y-8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X8.00000 Y8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X-8.00000 Y-8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X8.00000 Y8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X-8.00000 Y-8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X8.00000 Y8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X-8.00000 Y-8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X8.00000 Y8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X-8.00000 Y-8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X8.00000 Y8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X-8.00000 Y-8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X8.00000 Y8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X-8.00000 Y-8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X8.00000 Y8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X-8.00000 Y-8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X8.00000 Y8.00000 Z0.0000

In [155]:
g_code_end = "G0 X-10 Y-10 Z5\nG90; Set to Absolute Positioning\n"

In [156]:
s = gcode_initial+g_code_path+g_code_end
f = open('square.g','w')
f.write(s)
f.close()
print(s)

G92 Z0;set current z as z=0
G1 F1000; set feedrate
G91; Set to Relative Positioning 
T2;set tool
G1 X8.00000 Y8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X-8.00000 Y-8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X8.00000 Y8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X-8.00000 Y-8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X8.00000 Y8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X-8.00000 Y-8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X8.00000 Y8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X-8.00000 Y-8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X8.00000 Y8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X-8.00000 Y-8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X8.00000 Y8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X-8.00000 Y-8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X8.00000 Y8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X-8.00000 Y-8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X8.00000 Y8.00000 Z0.00000 E0.12189:0.00000 F1000.00000
G1 X

visualize your code here: https://ncviewer.com/